In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import cv2

In [2]:
np.random.seed(30)
import tensorflow as tf
tf.set_random_seed(30)

In [3]:
train_doc = np.random.permutation(open('notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 10

In [4]:
x_dim = 120
y_dim = 120
video_frame_len = 30
num_channel = 3

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    return np.float32([[1, 0, dx], [0, 1, dy]])

In [6]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, video_frame_len, x_dim, y_dim, num_channel)) 
    batch_labels = np.zeros((batch_size,5))
    return batch_data, batch_labels

In [7]:
def load_batch_data(source_path, folder_list, batch_num, batch_size, t):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    # We will also build a agumented batch data
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, video_frame_len)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (x_dim,y_dim), interpolation = cv2.INTER_AREA)
            batch_data[folder,idx] = resized
            batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))      

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
    return batch_data, batch_labels

In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_data(source_path, folder_list, batch, batch_size, t) 
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_data(source_path, folder_list, num_batches, batch_size, t)

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = 'notebooks/storage/Final_data/Collated_training/train'
val_path = 'notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [10]:
# Building the convolutional neural network architecture
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

filters = [8,16,32,64]
dense = [1000, 500, 5]

# Input Shape
input_shape=(video_frame_len,x_dim,y_dim,num_channel)

# Define model
model = Sequential()

model.add(Conv3D(filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(dense[2], activation='softmax'))

Using TensorFlow backend.


In [11]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [14]:
def get_steps_per_epoch(num_train_sequences, batch_size):
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1
    return steps_per_epoch
        
def get_validation_steps(num_val_sequences, batch_size):
    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
    return validation_steps

In [15]:
batch_size = 10
num_epochs = 10

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)


model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  notebooks/storage/Final_data/Collated_training/val ; batch size = 10
Source path =  notebooks/storage/Final_data/Collated_training/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 150s 2s/step - loss: 1.7804 - categorical_accuracy: 0.3032 - val_loss: 1.2845 - val_categorical_accuracy: 0.3950

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-1.78592-0.29940-1.28447-0.39500.h5
Epoch 2/10
67/67 [==============================] - 22s 332ms/step - loss: 1.3168 - categorical_accuracy: 0.4279 - val_loss: 1.6756 - val_categorical_accuracy: 0.3350

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-1.31679-0.42786-1.67564-0.33500.h5
Epoch 3/10
67/67 [==============================] - 23s 338ms/step - loss: 1.3250 - categorical_accuracy: 0.4925 - val_loss: 1.2428 - val_categorical_accuracy: 0.4850

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-1.32496-0.49254-1.24281-

In [16]:
batch_size = 20
num_epochs = 20

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 12s 359ms/step - loss: 0.5376 - categorical_accuracy: 0.8382 - val_loss: 0.9105 - val_categorical_accuracy: 0.7400

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.53756-0.83824-0.91054-0.74000.h5
Epoch 2/20
34/34 [==============================] - 11s 315ms/step - loss: 0.5749 - categorical_accuracy: 0.7696 - val_loss: 0.8701 - val_categorical_accuracy: 0.6800

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.57485-0.76961-0.87014-0.68000.h5
Epoch 3/20
34/34 [==============================] - 10s 309ms/step - loss: 0.5424 - categorical_accuracy: 0.7696 - val_loss: 0.9555 - val_categorical_accuracy: 0.6300

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.54236-0.76961-0.95555-0.63000.h5
Epoch 4/20
34/34 [==============================] - 10s 295ms/step - loss: 0.4884 - categorical_accuracy: 0.8039 - val_loss: 0.6315 - val_categorical_accurac

In [17]:
batch_size = 30
num_epochs = 30

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 8s 358ms/step - loss: 0.2277 - categorical_accuracy: 0.8986 - val_loss: 0.6190 - val_categorical_accuracy: 0.7750

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.22770-0.89855-0.61897-0.77500.h5
Epoch 2/30
23/23 [==============================] - 8s 361ms/step - loss: 0.2044 - categorical_accuracy: 0.9275 - val_loss: 0.6588 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.20444-0.92754-0.65876-0.80000.h5
Epoch 3/30
23/23 [==============================] - 7s 324ms/step - loss: 0.2590 - categorical_accuracy: 0.9058 - val_loss: 0.9764 - val_categorical_accuracy: 0.7000

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.25904-0.90580-0.97645-0.70000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 4/30
23/23 [==============================] - 7s 309ms/step - loss: 0.232

23/23 [==============================] - 7s 312ms/step - loss: 0.2062 - categorical_accuracy: 0.9420 - val_loss: 0.3982 - val_categorical_accuracy: 0.8375

Epoch 00027: saving model to model_init_2019-12-2217_53_53.115345/model-00027-0.20625-0.94203-0.39819-0.83750.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.907348723406699e-09.
Epoch 28/30
23/23 [==============================] - 7s 295ms/step - loss: 0.2869 - categorical_accuracy: 0.8841 - val_loss: 0.6114 - val_categorical_accuracy: 0.8125

Epoch 00028: saving model to model_init_2019-12-2217_53_53.115345/model-00028-0.28692-0.88406-0.61141-0.81250.h5
Epoch 29/30
23/23 [==============================] - 7s 320ms/step - loss: 0.1815 - categorical_accuracy: 0.9565 - val_loss: 0.4929 - val_categorical_accuracy: 0.8125

Epoch 00029: saving model to model_init_2019-12-2217_53_53.115345/model-00029-0.18147-0.95652-0.49293-0.81250.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 9.536743617033494e-10.
Epoch 3

In [18]:
batch_size = 50
num_epochs = 50

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
14/14 [==============================] - 6s 398ms/step - loss: 0.2407 - categorical_accuracy: 0.9167 - val_loss: 0.1838 - val_categorical_accuracy: 0.9250

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.24069-0.91667-0.18381-0.92500.h5
Epoch 2/50
14/14 [==============================] - 4s 318ms/step - loss: 0.1808 - categorical_accuracy: 0.9286 - val_loss: 1.3236 - val_categorical_accuracy: 0.6750

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.18077-0.92857-1.32362-0.67500.h5
Epoch 3/50
14/14 [==============================] - 5s 367ms/step - loss: 0.1724 - categorical_accuracy: 0.9524 - val_loss: 0.4642 - val_categorical_accuracy: 0.9000

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.17237-0.95238-0.46421-0.90000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.768371808516747e-10.
Epoch 4/50
14/14 [==============================] - 4s 273ms/step - loss: 0.246

14/14 [==============================] - 4s 317ms/step - loss: 0.1835 - categorical_accuracy: 0.9286 - val_loss: 0.3700 - val_categorical_accuracy: 0.9500

Epoch 00027: saving model to model_init_2019-12-2217_53_53.115345/model-00027-0.18349-0.92857-0.37001-0.95000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.164153273563659e-13.
Epoch 28/50
14/14 [==============================] - 4s 313ms/step - loss: 0.1646 - categorical_accuracy: 0.9048 - val_loss: 0.7011 - val_categorical_accuracy: 0.5500

Epoch 00028: saving model to model_init_2019-12-2217_53_53.115345/model-00028-0.16460-0.90476-0.70114-0.55000.h5
Epoch 29/50
14/14 [==============================] - 4s 296ms/step - loss: 0.1566 - categorical_accuracy: 0.9405 - val_loss: 0.8668 - val_categorical_accuracy: 0.7000

Epoch 00029: saving model to model_init_2019-12-2217_53_53.115345/model-00029-0.15664-0.94048-0.86677-0.70000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 5.820766367818295e-14.
Epoch 3

In [19]:
batch_size = 90
num_epochs = 30

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 3s 393ms/step - loss: 0.2356 - categorical_accuracy: 0.8750 - val_loss: 0.4571 - val_categorical_accuracy: 0.8250

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.23558-0.87500-0.45708-0.82500.h5
Epoch 2/30
8/8 [==============================] - 2s 300ms/step - loss: 0.3128 - categorical_accuracy: 0.9167 - val_loss: 1.0495 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.31282-0.91667-1.04950-0.55000.h5
Epoch 3/30
8/8 [==============================] - 2s 304ms/step - loss: 0.2984 - categorical_accuracy: 0.8333 - val_loss: 1.0629 - val_categorical_accuracy: 0.7750

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.29839-0.83333-1.06293-0.77500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.842171078036277e-17.
Epoch 4/30
8/8 [==============================] - 2s 308ms/step - loss: 0.2169 - cate

8/8 [==============================] - 3s 337ms/step - loss: 0.2182 - categorical_accuracy: 0.9167 - val_loss: 0.5327 - val_categorical_accuracy: 0.7750

Epoch 00027: saving model to model_init_2019-12-2217_53_53.115345/model-00027-0.21818-0.91667-0.53271-0.77500.h5
Epoch 28/30
8/8 [==============================] - 3s 353ms/step - loss: 0.3952 - categorical_accuracy: 0.8125 - val_loss: 0.8188 - val_categorical_accuracy: 0.6500

Epoch 00028: saving model to model_init_2019-12-2217_53_53.115345/model-00028-0.39524-0.81250-0.81884-0.65000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.3877788466974007e-20.
Epoch 29/30
8/8 [==============================] - 3s 315ms/step - loss: 0.1289 - categorical_accuracy: 0.9583 - val_loss: 0.3611 - val_categorical_accuracy: 0.8000

Epoch 00029: saving model to model_init_2019-12-2217_53_53.115345/model-00029-0.12894-0.95833-0.36107-0.80000.h5
Epoch 30/30
8/8 [==============================] - 3s 317ms/step - loss: 0.2252 - categorical

In [20]:
batch_size = 90
num_epochs = 10


steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
8/8 [==============================] - 3s 417ms/step - loss: 0.5060 - categorical_accuracy: 0.7500 - val_loss: 0.6101 - val_categorical_accuracy: 0.7000

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.50602-0.75000-0.61010-0.70000.h5
Epoch 2/10
8/8 [==============================] - 3s 355ms/step - loss: 0.4179 - categorical_accuracy: 0.8333 - val_loss: 0.7778 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.41786-0.83333-0.77781-0.75000.h5
Epoch 3/10
8/8 [==============================] - 3s 339ms/step - loss: 0.3372 - categorical_accuracy: 0.8958 - val_loss: 0.3420 - val_categorical_accuracy: 0.8750

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.33725-0.89583-0.34196-0.87500.h5
Epoch 4/10
8/8 [==============================] - 2s 305ms/step - loss: 0.1268 - categorical_accuracy: 0.9792 - val_loss: 0.6963 - val_categorical_accuracy: 0.8500

E

In [21]:
batch_size = 90
num_epochs = 20


steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 3s 375ms/step - loss: 0.1976 - categorical_accuracy: 0.9167 - val_loss: 0.5237 - val_categorical_accuracy: 0.7750

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.19759-0.91667-0.52372-0.77500.h5
Epoch 2/20
8/8 [==============================] - 3s 327ms/step - loss: 0.1636 - categorical_accuracy: 0.9167 - val_loss: 0.6546 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.16357-0.91667-0.65456-0.70000.h5
Epoch 3/20
8/8 [==============================] - 3s 313ms/step - loss: 0.1945 - categorical_accuracy: 0.9583 - val_loss: 1.5092 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.19447-0.95833-1.50915-0.50000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 8.673617791858755e-22.
Epoch 4/20
8/8 [==============================] - 3s 351ms/step - loss: 0.4007 - cate

In [22]:
batch_size = 90
num_epochs = 5

steps_per_epoch = get_steps_per_epoch(num_train_sequences, batch_size)
validation_steps = get_validation_steps(num_val_sequences, batch_size)
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5
8/8 [==============================] - 3s 424ms/step - loss: 0.2020 - categorical_accuracy: 0.9583 - val_loss: 0.8965 - val_categorical_accuracy: 0.7000

Epoch 00001: saving model to model_init_2019-12-2217_53_53.115345/model-00001-0.20201-0.95833-0.89654-0.70000.h5
Epoch 2/5
8/8 [==============================] - 3s 385ms/step - loss: 0.3322 - categorical_accuracy: 0.8750 - val_loss: 0.8260 - val_categorical_accuracy: 0.6000

Epoch 00002: saving model to model_init_2019-12-2217_53_53.115345/model-00002-0.33221-0.87500-0.82597-0.60000.h5
Epoch 3/5
8/8 [==============================] - 3s 321ms/step - loss: 0.1964 - categorical_accuracy: 0.8958 - val_loss: 0.5169 - val_categorical_accuracy: 0.8750

Epoch 00003: saving model to model_init_2019-12-2217_53_53.115345/model-00003-0.19640-0.89583-0.51693-0.87500.h5
Epoch 4/5
8/8 [==============================] - 3s 313ms/step - loss: 0.1336 - categorical_accuracy: 0.9375 - val_loss: 0.5581 - val_categorical_accuracy: 0.7750

Epoch

#### The accuracy of the training set and the validation set are 0.9167 and 0.8500 respectively.